# Tomographic Reconstruction: Centre of Rotation

For a successful reconstruction we need `matplotlib` for visualisation and debugging, `numpy` for array manipulation, `tomopy` so we can import `tomocuda`, `tomocuda` to run correction algorithms on a GPU, write_centerfrom `tomopy.recon.rotation` to write individual slices to disk with varying centres of rotation, and the `algorithm` function from `tomopy.recon.algorithm`. We take `dxchange` to read and write data from and to disk and `datetime` for benchmarking our processing speed, `os` is used for `os.path.join`.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import tomopy
import tomocuda
from tomopy.recon.rotation import write_center
from tomopy.recon.algorithm import recon
import dxchange as tir
import datetime
import os

We start by setting some basic directory information for input and output. An unreconstructed tomographic image series is required `file` (radiographs of the sample through 360 deg.), and a dark current `file_dark` and open beam `file_flat` image. These are contained within `data_dir`.

In [ ]:
data_dir  = '' # Input data directory
file      = '' # Input file in data directory
file_flat = '' # Open beam file
file_dark = '' # Dark current file

output_dir  = data_dir
file_name   = os.path.join(data_dir, file)
flat_name   = os.path.join(data_dir, file_flat)
dark_name   = os.path.join(data_dir, data_dark)
output_file = '{}/recon_{}/recon_{}_'.format(output_dir, file.split(".")[-2], file.split(".")[-2])

Now we can load our data into memory using the `tir` module. The 2-BM instrument provides data in the HDF5 file format, so we use the appropriate function to load the file. `data` contains our raw input file, the radiographic series. `white` contains out open beam data, and `dark` contains the dark current image. NOTE: the 2-BM instrument is configured to write data to `exchange/data_dark` regardless of the acquisition type, this is not an error. The volume is cropped here; the full height of the radiograph is not required until a full reconstruction is run. This makes the process more rapid and reduces the memory requirement.

Here, we are taking the third tomographic scan in this series. In each scan we write 600 radiographs to the hard disk (start 0 to 599, 600 to 1200, 1200 to 1800, 1800 to 2400 and so on and so forth).

In [ ]:
data  = tir.read_hdf5(file_name, 'exchange/data_dark', slc=((1800,2400), (700,720,1)))
white = tir.read_hdf5(flat_mame, 'exchange/data_dark', slc=((1,9),       (700,720,1)))
dark  = tir.read_hdf5(dark_name, 'exchange/data_dark', slc=((1,9),       (700,720,1)))

The shape of the data set is required, and the theta value for each rotational position is determined using the numpy `linspace` function.

In [ ]:
data_size = data.shape
theta     = np.linspace(0, np.pi, num=data_size[0]) 

Now outliers are removed from the raw data and open beam acquisitions. Outliers are not removed from the dark current data as these are outliers by definition. This is carried out using the `tomocuda` package which uses the NVIDIA CUDA interface. The outlier level is defined as 200. This is a slow operation, and we can optionally save to disk here to prevent us having to re-execute this if a mistake is made further on. The following cell can re-load the data if required.

In [ ]:
outlier_level = 200
data  = tomocuda.remove_outlier_cuda(data,  outlier_level, size=15)
white = tomocuda.remove_outlier_cuda(white, outlier_level, size=15)

# np.savez('data.npz', data)
# np.savez('white.npz', white)

In [ ]:
# data  = np.load('data.npz')
# white = np.load('white.npz')

Data are now corrected for scintillator artefacts using the open beam signal and systematic (non-variable) noise using the dark current data. The first slice becomes corrupt by this so we remove it (reason unknown, probably the source code of tomopy).

In [ ]:
data = tomopy.prep.normalize(data, white, dark) # Correct for dark current and open beam
data = tomopy.prep.normalize_bg(data, air=10)   # Normalize the background to 10
data[0,:,:] = data[1,:,:]                       # Remove the corrupted first slice 

Stripes are now removed using the Fourier Wavelet method. This is rapid; we can do this using the CPU. `level` is the number of discrete transformatio  levels, `wname` is the type of filter, `sigma` is the damping parameter, `pad` pads the sinogram with zeros. Run on 4 cores.

In [ ]:
data = tomopy.prep.stripe.remove_stripe_fw(data,
                                           level = 6,
                                           wname = 'sym16'
                                           sigma = 2,
                                           pad   = True,
                                           ncore = 4)

Now the phase contrast is extracted and merged with the absorption contrast by the regularisation parameter set in `rat`. This is set, and then left identical for <b>all</b> reconstructions we are carrying out to ensure a consistent method. The array is padded with zeros, we've got a 27 keV incident beam and the PCO Dimax camera has a pixel size of 11 um. Our propagation length is 10 cm.

In [ ]:
rat        = 0.1e-2 # Regularisation Parameter
pixel_size = 0.0011 # Detector Pixel Size in cm (PCO Dimax)
eng        = 27     # Incident beam energy in keV
z          = 10     # Wavefront Propagation Distance in cm

data = tomopy.prep.phase.retrieve_phase(data, 
                                        pixel_size=pxl, 
                                        dist=z, 
                                        energy=eng, 
                                        alpha=rat, 
                                        pad=True,
                                        ncore=4)

We are now ready to reconstruct and write the data to disk. we only need a single slice to reconstruct and find the center of rotation, so we trim our dataset down once again to this. The `cen_range` tuple is the range of values over which the center of rotation is expected, approximately half way through the image, and the delta value which sets our precision. Once written to disk this can be loaded into a package such as imagej and searched for the correct center of rotation value.

In [ ]:
write_center(data[:,19:21,:], theta, dpath=output_dir, cen_range=(2000, 2500, 10))

Following this, we can adjust the `cen_range` parameter and refine further and further down to give a more and more precise location.